In [ ]:
!pip install pyarrow
!pip install datasets evaluate
!pip install transformers==4.28.0
!pip install thefuzz
!pip install dedupe
!pip install pandas_dedupe


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from google.colab import drive
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from thefuzz import fuzz
from thefuzz import process
import dedupe
import pandas_dedupe
import json

In [ ]:
complete_food_data = pd.read_excel('/content/data_bulk_raw.xlsx')

In [ ]:
complete_food_data

,Unique Id,Food Product Category,Origin Detail,Level of Processing,Primary Food Product Category,Product Type,# of Units,Volume Per Unit,Total Volume in lbs,Average Cost Per Unit,Total Cost,Nutrition Certifications,Product Name,Food Product Group,Primary Food Product Group,Unnamed: 15
0,gtla-18-allpurchases-10,Beef,Newport Meats,Ultra-Processed,Beef,beef patty 2 oz,NaN,NaN,NaN,50.000,3450.00,Red Meat,"beef, patty",Meat,Meat,NaN
1,gtla-18-allpurchases-3,Fruit,SAGE FRUIT COMPANY,Whole/Minimally Processed,Fruit,APPLE-FUJI (80/90 CT) SORT 5 LB,NaN,NaN,NaN,8.300,16.60,NaN,"apple, fuji",Produce,Produce,NaN
2,gtla-18-allpurchases-16,Fruit,"BLAZER WILKINSON, L.P.",Whole/Minimally Processed,Fruit,BERRY-STRAWBERRY LAS CASE,NaN,NaN,NaN,18.420,406.20,NaN,strawberry,Produce,Produce,NaN
3,gtla-18-allpurchases-21,Fruit,DAVID OPPENHEIMER & COMPANY,Whole/Minimally Processed,Fruit,BERRY-STRAWBERRY LAS CASE,NaN,NaN,NaN,18.850,120.70,NaN,strawberry,Produce,Produce,NaN
4,wccusd-1718-ll-82919-57,Condiments & Snacks,Buena Vista,Ultra-Processed,Condiments & Snacks,Muffin Top Sweet Potato Choc 63218,201,NaN,4896.00000,52.240,10500.52,Grain Based,"muffin top, sweet potato chocolate",Condiments & Snacks,Condiments & Snacks,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154593,scvmc-2122-pc-122022-2017,Yogurt,"FRESH & READY, San Fernando, CA",Ultra-Processed,Yogurt,"FR, BLUEBERRY PARFAIT",1,0.46875,0.46875,3.294,3.29,NaN,"yogurt, parfait, blueberry",Milk & Dairy,Milk & Dairy,NaN
154594,scvmc-2122-pc-122022-2018,Yogurt,"FRESH & READY, San Fernando, CA",Ultra-Processed,Yogurt,"FR, STRAWBERRY PARFAIT",1,0.46875,0.46875,3.152,3.15,NaN,"yogurt, parfait, strawberry",Milk & Dairy,Milk & Dairy,NaN
154595,scvmc-2122-pc-122022-2019,Yogurt,"FRESH & READY, San Fernando, CA",Ultra-Processed,Yogurt,"FR, MIXED BERRY PARFAIT",1,0.46875,0.46875,3.294,3.29,NaN,"yogurt, parfait, mixed berry",Milk & Dairy,Milk & Dairy,NaN
154596,scvmc-2122-pc-122022-2020,Yogurt,DANONE US LLC,Ultra-Processed,Yogurt,"YOGURT, VNL GREEK SS CUP REF",1490,3.97,5915.30000,11.208,16710.72,NaN,"yogurt, flavored, ss",Milk & Dairy,Milk & Dairy,NaN


In [ ]:
def preprocess_function(examples):
    """
    Preprocesses the input examples using the tokenizer.

    Args:
        examples (dict): A dictionary containing the input examples.

    Returns:
        dict: A dictionary containing the preprocessed examples.
    """
    return tokenizer(examples["text"], truncation=True)

def compute_metrics(eval_pred):
    """
    Computes the evaluation metrics based on the evaluation predictions.

    Args:
        eval_pred (tuple): A tuple containing the evaluation predictions and labels.

    Returns:
        dict: A dictionary containing the computed evaluation metrics.
    """
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
class DataSet():

    def __init__(self, dataframe):
        '''
        attributes:
        self.data (dataframe)
        self.dataset_observations (int)
        self.matching_fucntion (function)
        self.match_threshold (int)
        self.labels_grouping (dict)
        self.labels_set (set)
        self.columns_list (list)
        self.renaming_mapping (dict)
        self.add_list (list)
        self.rm_list (list)
        self.label_correspondence (dict)
        self.reversed_label_correspondence (dict)
        self.label_mapping (???)
        self.id2label (json)
        self.label2id (json)
        '''
        self.data = dataframe
        self.dataset_observations = len(self.data)
        self.name = None

    def create_ss_column(self, new_column, extract_column):
        #complete_food_data[new_column] = complete_food_data[extract_column].str.lower().str.split(',').str[-1].str.strip().apply(lambda x: 'SS' if x == 'ss' else 'NA')
        complete_food_data[new_column] = complete_food_data[extract_column].apply(lambda x: 'SS' if isinstance(x, str) and (' ss' in x.lower() or ' ss ' in x.lower()) else 'NA')

    def create_basic_type_column(self, new_column, extract_column):
        self.data[new_column] = self.data[extract_column].str.lower().str.split(',').str[0].str.strip()

    def clean_data(self, columns_list: list, renaming_mapping: dict) -> pd.DataFrame:

        self.columns_list = columns_list
        self.renaming_mapping = renaming_mapping

        self.data = self.data[columns_list]
        self.data.rename(columns=renaming_mapping, inplace=True)

        for column in list(renaming_mapping.values()):
            self.data[column] = self.data[column].str.lower().str.replace(',', '').str.strip()
            self.data[column] = self.data[column].str.replace('prequalified: ', '')
            self.data.dropna(inplace=True)
            self.data.reset_index(drop=True, inplace=True)

        self.data = self.data.drop_duplicates().reset_index(drop=True)

    def group_labels(self, category: str, category_label: str, labels_column = 'labels', matching_function= fuzz.ratio, thresh = 75, topset = False) -> dict:

        self.matching_function = matching_function
        self.match_threshold = thresh

        #subset dataframe by category
        if category == None or category_label == None:
            column_name = self.data[[labels_column]]
        else:
            column_name = self.data[self.data[category] == category_label][[labels_column]]

        column_name['value_counts_column'] = column_name[labels_column].map(column_name[labels_column].value_counts())
        column_name = column_name.sort_values('value_counts_column', ascending=False).reset_index(drop=True)

        column_name = column_name[[labels_column]].values.flatten().tolist()
        print('number of non unique entries in column', len(column_name))

        #Create list of dictionaries of subgroups
        label_groups = {}
        for label_entry in column_name:
            counter = 0
            for label_group_key in label_groups.keys():
                if matching_function(label_entry, label_group_key) >= thresh:
                    counter += 1
                    if label_entry != label_group_key:
                        label_groups[label_group_key].append(label_entry)
            if counter == 0:
                label_groups[label_entry] = []

        if topset:
            return label_groups
        else:
            self.labels_grouping = label_groups

    def create_label_groupings_dictionary(self, topset_column_name='food_category'):
        self.label_groupings_dictionary = {}

        for category_label in self.data[topset_column_name].unique():
            self.label_groupings_dictionary[category_label] = self.group_labels(category=topset_column_name, category_label=category_label, topset=True)

    def clean_labels(self, add_list, rm_list, labels_to_correct ='food_category', inforich= False):

        def alter_food_set(labels_set, add_list, rm_list):
            print('number of elements in food set before additions', len(labels_set))
            labels_set.update(add_list)
            labels_set.difference_update(rm_list)

            print('number of elements in food set after additions', len(labels_set))

        def add_and_remove_keys_from_dict(input_dict, keys_to_add, keys_to_remove, default_value = []):
            for key in keys_to_remove:
                if key in input_dict:
                    del input_dict[key]

            for key in keys_to_add:
                if key not in input_dict:
                    input_dict[key] = default_value

        def find_best_match(misspelled_value, choices):
            best_match = process.extractOne(misspelled_value, choices)
            return best_match[0]

        self.add_list = add_list
        self.rm_list = rm_list
        self.labels_set = set(self.labels_grouping)

        alter_food_set(self.labels_set, self.add_list, self.rm_list)
        add_and_remove_keys_from_dict(self.labels_grouping, self.add_list, self.rm_list)

        self.data['corrected_labels'] = self.data['labels'].apply(lambda x: find_best_match(x, self.labels_set))

        if inforich:
            self.data[labels_to_correct] = self.data[labels_to_correct].apply(lambda x: find_best_match(x, self.labels_set))
            self.data = self.data[['text', 'food_category', 'labels']]
        else:
            self.data['corrected_labels'] = self.data['labels'].apply(lambda x: find_best_match(x, self.labels_set))
            self.data = self.data[['text', 'corrected_labels']]
            self.data.rename(columns={'corrected_labels': 'labels'}, inplace=True)

    def factorize_dataset(self):
        self.data['labels'], self.label_mapping = pd.factorize(self.data['labels'])
        label_correspondence = dict(zip(self.data['labels'].unique(), self.label_mapping))
        self.id2label = {int(key): value for key, value in label_correspondence.items()}
        self.label2id = {value: key for key, value in self.id2label.items()}

    def shuffle_dataset(self):
        self.data = shuffle(self.data)
        self.data.reset_index(drop=True, inplace=True)

def sweep_data(dataframe, add_list, rm_list, columns_list, renaming_mapping, category=None, category_label=None, thresh=85, match_function=fuzz.ratio, labels_column = 'labels', new=False):
    dataset = DataSet(dataframe)

    if new == 'Basic Type':
        dataset.create_basic_type_column('Basic Type', 'Product Name')

    if new == 'Serving Size':
        dataset.create_ss_column('Serving Size', 'Product Name')

    dataset.clean_data(columns_list, renaming_mapping)
    dataset.group_labels(category=category ,category_label=category_label, thresh=thresh, matching_function=match_function, labels_column=labels_column)
    dataset.clean_labels(add_list, rm_list)
    dataset.factorize_dataset()
    return dataset

def create_food_dataset(dataframe=complete_food_data, columns_list=columns_list2, renaming_mapping=renaming_mapping2, add_list=food_add_list, rm_list = food_rm_list, category=None, category_label=None, thresh=85, match_function=fuzz.ratio, labels_column= 'food_category'):

    food_dataset = DataSet(dataframe)
    food_dataset.clean_data(columns_list, renaming_mapping)
    food_dataset.group_labels(category=category ,category_label=category_label, thresh=thresh, matching_function=match_function, labels_column= labels_column)
    food_dataset.clean_labels(add_list, rm_list, inforich=True)

    return food_dataset

def create_datasets(food_dataset):
    DataSets_list = []
    DataSet2id = {}
    for index, name in enumerate(food_dataset.labels_set):
        dataset = DataSet(food_dataset.data[food_dataset.data['food_category'] == list(food_dataset.labels_set)[index]])
        dataset.name = name
        dataset.data.name = name
        DataSets_list.append(dataset)
        DataSet2id[name] = index

    return DataSets_list, DataSet2id

def clean_datasets(DataSets_list, DataSet2id, total_add_list, total_rm_list, new):
    cleaned_datasets = []
    ClnDataSet2id = {}
    for index, dataset in enumerate(DataSets_list):
        test_columns_list = ['text', 'labels']
        test_renaming_mapping = {}
        cleaned_dataset = sweep_data(dataset.data, total_add_list[DataSet2id[dataset.name]], total_rm_list[DataSet2id[dataset.name]], test_columns_list, test_renaming_mapping, new)
        cleaned_dataset.name = dataset.name
        cleaned_dataset.data.name = dataset.name
        cleaned_datasets.append(cleaned_dataset)
        ClnDataSet2id[dataset.name] = index

    return cleaned_datasets, ClnDataSet2id

def create_and_populate_add_rm_lists(food_dataset):
    total_add_list = []
    total_rm_list = []

    n = 0
# eggs 0 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_24 = []
    rm_24 = []
    total_add_list.append(add_24)
    total_rm_list.append(rm_24)

# produce 11 check ################################################################################
    # print(list(food_dataset.labels_set)[0])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[0]])
    # add_0 = []
    # rm_0 = []
    # total_add_list.append(add_0)
    # total_rm_list.append(rm_0)

    n = 1
# fish (farm-raised) 1 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_7 = []
    rm_7 = ['sea bass']
    total_add_list.append(add_7)
    total_rm_list.append(rm_7)

# grain products 10 check ################################################################################
    # print(list(food_dataset.labels_set)[1])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[1]])
    # add_1 = ['toast','egg barley', 'corn']
    # rm_1 = []
    # total_add_list.append(add_1)
    # total_rm_list.append(rm_1)

    n = 2
# seafood 2 check ################################################################################
    print(list(food_dataset.labels_set)[4])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[4]])
    add_4 = ['crawfish']
    rm_4 = ['sea bass']
    total_add_list.append(add_4)
    total_rm_list.append(rm_4)

# beef 3 check ################################################################################
    # print(list(food_dataset.labels_set)[2])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[2]])
    # add_2 = ['gyro meat', 'entree']
    # rm_2 = ['gryo meat', 'entrée']
    # total_add_list.append(add_2)
    # total_rm_list.append(rm_2)

    n = 3
# beef 3 check ################################################################################
    print(list(food_dataset.labels_set)[2])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[2]])
    add_2 = ['gyro meat', 'entree']
    rm_2 = ['gryo meat', 'entrée']
    total_add_list.append(add_2)
    total_rm_list.append(rm_2)

# pork 14 check ################################################################################
    # print(list(food_dataset.labels_set)[3])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[3]])
    # add_3 = []
    # rm_3 = []
    # total_add_list.append(add_3)
    # total_rm_list.append(rm_3)

    n = 4
# cheese 4 check  ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_20 = []
    rm_20 = []
    total_add_list.append(add_20)
    total_rm_list.append(rm_20)

# seafood 2 check ################################################################################
    # print(list(food_dataset.labels_set)[4])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[4]])
    # add_4 = ['crawfish']
    # rm_4 = ['sea bass']
    # total_add_list.append(add_4)
    # total_rm_list.append(rm_4)

    n = 5
# turkey other poultry 5 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_19 = ['turkey ham']
    rm_19 = []
    total_add_list.append(add_19)
    total_rm_list.append(rm_19)

# yogurt 23 check ################################################################################
    # print(list(food_dataset.labels_set)[5])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[5]])
    # add_5 = []
    # rm_5 = []
    # total_add_list.append(add_5)
    # total_rm_list.append(rm_5)

    n = 6
# tree nuts & seeds 6 check  ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_14 = []
    rm_14 = []
    total_add_list.append(add_14)
    total_rm_list.append(rm_14)

# non-food 25 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_6 = []
    # rm_6 = []
    # total_add_list.append(add_6)
    # total_rm_list.append(rm_6)

    n = 7
# roots & tubers 7 check  ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_12 = ['vegetable']
    rm_12 = []
    total_add_list.append(add_12)
    total_rm_list.append(rm_12)

# fish (farm-raised) 1 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_7 = []
    # rm_7 = ['sea bass']
    # total_add_list.append(add_7)
    # total_rm_list.append(rm_7)

    n = 8
# meals 8 check  ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_11 = ['entree', 'pasta salad', 'scrapple','mozzarella stick', 'pretzel dog', 'baked potato', 'egg substitute',
            'egg alternative','oat','breakfast']
    rm_11 = ['entrée','breakfast stick']
    total_add_list.append(add_11)
    total_rm_list.append(rm_11)

# milk 22 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_8 = []
    # rm_8 = []
    # total_add_list.append(add_8)
    # total_rm_list.append(rm_8)

    n = 9
# fruit 9 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_21 = ['raspberry', 'cranberries', ]
    rm_21 = []
    total_add_list.append(add_21)
    total_rm_list.append(rm_21)

# rice 20 checkk ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_9 = []
    # rm_9 = []
    # total_add_list.append(add_9)
    # total_rm_list.append(rm_9)

    n = 10
# grain products 10 check ################################################################################
    print(list(food_dataset.labels_set)[1])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[1]])
    add_1 = ['toast','egg barley', 'corn']
    rm_1 = []
    total_add_list.append(add_1)
    total_rm_list.append(rm_1)

# condiments & snacks 19 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_10 = ['pie','doughnut','muffin top','seasoning salt','seasoning tajin','peppercorn', 'tomatillo', 'malt','butter',
    #    'baking soda','sherbet','fruit cup','corn','scone','cereal']
    # rm_10 = ['sherbert']
    # total_add_list.append(add_10)
    # total_rm_list.append(rm_10)

    n = 11
# produce 11 check ################################################################################
    print(list(food_dataset.labels_set)[0])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[0]])
    add_0 = []
    rm_0 = []
    total_add_list.append(add_0)
    total_rm_list.append(rm_0)

# meals 8 check  ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_11 = ['entree', 'pasta salad', 'scrapple','mozzarella stick', 'pretzel dog', 'baked potato', 'egg substitute',
    #         'egg alternative','oat','breakfast']
    # rm_11 = ['entrée','breakfast stick']
    # total_add_list.append(add_11)
    # total_rm_list.append(rm_11)

    n = 12
# legumes 12 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_17 = []
    rm_17 = []
    total_add_list.append(add_17)
    total_rm_list.append(rm_17)

# roots & tubers 7 check  ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_12 = ['vegetable']
    # rm_12 = []
    # total_add_list.append(add_12)
    # total_rm_list.append(rm_12)

    n = 13
# meat 13 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_23 = []
    rm_23 = []
    total_add_list.append(add_23)
    total_rm_list.append(rm_23)

# vegetables 15 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_13 = ['tomatillo','vegetable blend','vegetable cup','vegetable mix','broccolini','broccoli rabe', 'green pea',
    #         'brussels sprout', 'romaine', 'ti leaf']
    # rm_13 = ['brussel sprout']
    # total_add_list.append(add_13)
    # total_rm_list.append(rm_13)

    n = 14
# pork 14 check ################################################################################
    print(list(food_dataset.labels_set)[3])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[3]])
    add_3 = []
    rm_3 = []
    total_add_list.append(add_3)
    total_rm_list.append(rm_3)

# tree nuts & seeds 6 check  ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_14 = []
    # rm_14 = []
    # total_add_list.append(add_14)
    # total_rm_list.append(rm_14)

    n = 15
# vegetables 15 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_13 = ['tomatillo','vegetable blend','vegetable cup','vegetable mix','broccolini','broccoli rabe', 'green pea',
            'brussels sprout', 'romaine', 'ti leaf']
    rm_13 = ['brussel sprout']
    total_add_list.append(add_13)
    total_rm_list.append(rm_13)

# chicken 24 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_15 = []
    # rm_15 = []
    # total_add_list.append(add_15)
    # total_rm_list.append(rm_15)

    n = 16
# fish (wild) 16 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_22 = []
    rm_22 = []
    total_add_list.append(add_22)
    total_rm_list.append(rm_22)

# butter 21 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_16 = []
    # rm_16 = []
    # total_add_list.append(add_16)
    # total_rm_list.append(rm_16)

    n = 17
# beverages 17 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_25 = []
    rm_25 = []
    total_add_list.append(add_25)
    total_rm_list.append(rm_25)

# legumes 12 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_17 = []
    # rm_17 = []
    # total_add_list.append(add_17)
    # total_rm_list.append(rm_17)

    n = 18
# milk & dairy 18 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_18 = ['butter', 'cream']
    rm_18 = []
    total_add_list.append(add_18)
    total_rm_list.append(rm_18)

    n = 19
# condiments & snacks 19 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_10 = ['pie','doughnut','muffin top','seasoning salt','seasoning tajin','peppercorn', 'tomatillo', 'malt','butter',
       'baking soda','sherbet','fruit cup','corn','scone','cereal']
    rm_10 = ['sherbert']
    total_add_list.append(add_10)
    total_rm_list.append(rm_10)

# turkey other poultry 5 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_19 = ['turkey ham']
    # rm_19 = []
    # total_add_list.append(add_19)
    # total_rm_list.append(rm_19)

    n = 20
# rice 20 checkk ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_9 = []
    rm_9 = []
    total_add_list.append(add_9)
    total_rm_list.append(rm_9)

# cheese 4 check  ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_20 = []
    # rm_20 = []
    # total_add_list.append(add_20)
    # total_rm_list.append(rm_20)

    n = 21
# butter 21 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_16 = []
    rm_16 = []
    total_add_list.append(add_16)
    total_rm_list.append(rm_16)

# fruit 9 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_21 = ['raspberry', 'cranberries', ]
    # rm_21 = []
    # total_add_list.append(add_21)
    # total_rm_list.append(rm_21)

    n = 22
# milk 22 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_8 = []
    rm_8 = []
    total_add_list.append(add_8)
    total_rm_list.append(rm_8)

# fish (wild) 16 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_22 = []
    # rm_22 = []
    # total_add_list.append(add_22)
    # total_rm_list.append(rm_22)

    n = 23
# yogurt 23 check ################################################################################
    print(list(food_dataset.labels_set)[5])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[5]])
    add_5 = []
    rm_5 = []
    total_add_list.append(add_5)
    total_rm_list.append(rm_5)

# meat 13 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_23 = []
    # rm_23 = []
    # total_add_list.append(add_23)
    # total_rm_list.append(rm_23)

    n = 24
# chicken 24 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_15 = []
    rm_15 = []
    total_add_list.append(add_15)
    total_rm_list.append(rm_15)

# eggs 0 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_24 = []
    # rm_24 = []
    # total_add_list.append(add_24)
    # total_rm_list.append(rm_24)

    n = 25
# non-food 25 check ################################################################################
    print(list(food_dataset.labels_set)[n])
    print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    add_6 = []
    rm_6 = []
    total_add_list.append(add_6)
    total_rm_list.append(rm_6)

# beverages 17 check ################################################################################
    # print(list(food_dataset.labels_set)[n])
    # print(food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]])
    # add_25 = []
    # rm_25 = []
    # total_add_list.append(add_25)
    # total_rm_list.append(rm_25)

    return total_add_list, total_rm_list

In [ ]:
print(list(food_dataset.labels_set))

NameError: ignored

In [ ]:
columns_list = ['Product Type', 'Food Product Category']
renaming_mapping = {'Product Type': 'text', 'Food Product Category': 'labels'}
fpg_add_list =[]
fpg_rm_list =['turkey', 'bread grains & legumes']

columns_list1 = ['Product Type', 'Primary Food Product Category']
renaming_mapping1 = {'Product Type': 'text', 'Primary Food Product Category': 'labels'}
add_list1=['meat']
rm_list1=['meats', 'turkey']

columns_list2 = ['Product Type', 'Food Product Category', 'Basic Type']
renaming_mapping2 = {'Product Type': 'text', 'Food Product Category': 'food_category', 'Basic Type': 'labels'}
food_add_list =[]
food_rm_list =['turkey']




In [ ]:
dataset = DataSet(complete_food_data)

In [ ]:
dataset.create_ss_column('Serving Size', 'Product Name')

In [ ]:
df = dataset.data
filtered_df = df[(df['Product Name'].str.contains('ss', case=False, na=False)) & (df['Serving Size'] == 'SS')]
filtered_df

In [ ]:
dataset.clean_data(['Product Type', 'Serving Size'], {'Product Type': 'text', 'Serving Size': 'labels'})

<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)
<ipython-input-9-b39de0d8c0e8>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column] = self.data[column].str.lower().str.replace(',', '').str.strip()
<ipython-input-9-b39de0d8c0e8>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [ ]:
ss_df = dataset.data

In [ ]:
dataset.data


,text,labels
0,beef patty 2 oz,na
1,apple-fuji (80/90 ct) sort 5 lb,na
2,berry-strawberry las case,na
3,muffin top sweet potato choc 63218,na
4,cheese-blue point reyes ~6 lb,na
...,...,...
80940,yogurt stwby blnded ss tube,ss
80941,fr blueberry parfait,na
80942,fr strawberry parfait,na
80943,fr mixed berry parfait,na


In [ ]:
dataset.factorize_dataset()

In [ ]:
dataset.shuffle_dataset()

In [ ]:
dataset.data

,text,labels
0,yogurt asst stwby & stwbybana 306746,1
1,corn cut frz,0
2,apples chl sl,0
3,100500 rice brn long parboiled pkg-24/2#,0
4,cucumber diced peeld 1/ bag 5 lbs,0
...,...,...
80940,egg whl w/citric acid bag,0
80941,ice cream cup brwne choc fdge,1
80942,2% white 1/2 pints 50/cs local,1
80943,coffee cafe kilimanjaro,0


In [ ]:
dataset = DataSet(complete_food_data)

if new == 'Serving Size':
    dataset.create_ss_column('Serving Size', 'Product Name')

dataset.clean_data(columns_list, renaming_mapping)
dataset.group_labels(category=category ,category_label=category_label, thresh=thresh, matching_function=match_function, labels_column=labels_column)
dataset.clean_labels(add_list, rm_list)
dataset.factorize_dataset()
dataset.shuffle_dataset()
return dataset

NameError: ignored

In [ ]:
fpg = sweep_data(complete_food_data, fpg_add_list, fpg_rm_list, columns_list, renaming_mapping, new='Basic Type')
pfpg = sweep_data(complete_food_data, add_list1, rm_list1, columns_list1, renaming_mapping1, new='Basic Type')
food_dataset = create_food_dataset()

<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)
<ipython-input-9-b39de0d8c0e8>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column] = self.data[column].str.lower().str.replace(',', '').str.strip()
<ipython-input-9-b39de0d8c0e8>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

number of non unique entries in column 80894
number of elements in food set before additions 27
number of elements in food set after additions 26


<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)
<ipython-input-9-b39de0d8c0e8>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column] = self.data[column].str.lower().str.replace(',', '').str.strip()
<ipython-input-9-b39de0d8c0e8>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

number of non unique entries in column 80480
number of elements in food set before additions 28
number of elements in food set after additions 28


<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)
<ipython-input-9-b39de0d8c0e8>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column] = self.data[column].str.lower().str.replace(',', '').str.strip()
<ipython-input-9-b39de0d8c0e8>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

number of non unique entries in column 76437
number of elements in food set before additions 26
number of elements in food set after additions 26


In [ ]:
food_dataset.create_label_groupings_dictionary()

number of non unique entries in column 2405
number of non unique entries in column 7534
number of non unique entries in column 19493
number of non unique entries in column 2310
number of non unique entries in column 9983
number of non unique entries in column 4101
number of non unique entries in column 43
number of non unique entries in column 4889
number of non unique entries in column 2448
number of non unique entries in column 5857
number of non unique entries in column 7793
number of non unique entries in column 198
number of non unique entries in column 1037
number of non unique entries in column 776
number of non unique entries in column 901
number of non unique entries in column 373
number of non unique entries in column 575
number of non unique entries in column 1623
number of non unique entries in column 807
number of non unique entries in column 212
number of non unique entries in column 560
number of non unique entries in column 913
number of non unique entries in column 116

In [ ]:
print(list(food_dataset.labels_set))

['condiments & snacks', 'legumes', 'grain products', 'butter', 'vegetables', 'meat', 'milk & dairy', 'fruit', 'cheese', 'non-food', 'chicken', 'milk', 'rice', 'yogurt', 'meals', 'seafood', 'produce', 'eggs', 'turkey other poultry', 'roots & tubers', 'beverages', 'beef', 'tree nuts & seeds', 'fish (wild)', 'pork', 'fish (farm-raised)']


In [ ]:
n = 18
print(list(food_dataset.labels_set)[n])
food_dataset.label_groupings_dictionary[list(food_dataset.labels_set)[n]]

turkey other poultry


{'turkey': ['turkey ham',
  'turkey ham',
  'turkey ham',
  'turkey ham',
  'turkey ham'],
 'duck': [],
 'goose': [],
 'jerky': [],
 'rabbit': [],
 'bacon': [],
 'bologna': [],
 'cornish hen': [],
 'sausage': [],
 'puree': [],
 'taco': [],
 'dug': []}

In [ ]:
total_add_list, total_rm_list = create_and_populate_add_rm_lists(food_dataset=food_dataset)
DataSets_list, DataSet2id = create_datasets(food_dataset=food_dataset)
cleaned_datasets, ClnDataSet2id = clean_datasets(DataSets_list=DataSets_list, DataSet2id=DataSet2id, total_add_list=total_add_list, total_rm_list=total_rm_list, new=False)

condiments & snacks
{'condiment': [], 'sauce': [], 'dessert': [], 'spice': ['pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie'], 'chip': [], 'dressing': [], 'cracker': [], 'snack': ['snacks'], 'bar': [], 'cookie': [], 'pastry': ['patry', 'patry'], 'dough': ['doughnut', 'doughnut', 'doughnut'], 'muffin': ['muffin top', 'muffin top', 'muffin top', 'muffin top', 'muffin top', 'muffin top'], 'mix': ['mx'], 'oil': [], 'seasoning': ['seasoning salt', 'seasoning salt', 'seasoning salt', 'seasoning salt', 'seasoning salt', 'seasoning salt', 'seasoning tajin', 'seasoning tajin'], 'candy': [], 'sugar': [], 'base': [], 'spread': [], 'vinegar': [], 'topping': [], 'juice': [], 'pepper': ['peppercorn'], 'salsa': [], 'herb': [], 'olive': [], 'creamer': ['cream'], 'tomato': ['tomatillo', 'tomatillo', 'tomatillo', 'tomatillo'], 'salt': ['malt'], 'paste': [], 'chocolate': [], 'margarine': [], 'honey': [], 'batter': ['butter', 'butter', 'butter', 'butt

<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)


number of elements in food set before additions 247
number of elements in food set after additions 247


<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)
<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)


number of non unique entries in column 807
number of elements in food set before additions 13
number of elements in food set after additions 13
number of non unique entries in column 5857
number of elements in food set before additions 74
number of elements in food set after additions 75


...

<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)


number of non unique entries in column 4889
number of elements in food set before additions 27
number of elements in food set after additions 27


<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)
<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)
<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)


number of non unique entries in column 2405
number of elements in food set before additions 20
number of elements in food set after additions 20
number of non unique entries in column 560
number of elements in food set before additions 13
number of elements in food set after additions 13
number of non unique entries in column 198
number of elements in food set before additions 21
number of elements in food set after additions 21
number of non unique entries in column 1623
number of elements in food set before additions 15
number of elements in food set after additions 15
number of non unique entries in column 102
number of elements in food set before additions 11
number of elements in food set after additions 11


<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)
<ipython-input-9-b39de0d8c0e8>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.rename(columns=renaming_mapping, inplace=True)


In [ ]:
notebook_login()

In [ ]:
fpg.data

,text,labels
0,beef patty 2 oz,0
1,apple-fuji (80/90 ct) sort 5 lb,1
2,berry-strawberry las case,1
3,muffin top sweet potato choc 63218,2
4,cheese-blue point reyes ~6 lb,3
...,...,...
80889,yogurt stwby blnded ss tube,21
80890,fr blueberry parfait,21
80891,fr strawberry parfait,21
80892,fr mixed berry parfait,21


In [ ]:
fpg.id2label

{0: 'beef',
 1: 'fruit',
 2: 'condiments & snacks',
 3: 'cheese',
 4: 'vegetables',
 5: 'roots & tubers',
 6: 'produce',
 7: 'beverages',
 8: 'chicken',
 9: 'grain products',
 10: 'meals',
 11: 'fish (wild)',
 12: 'milk & dairy',
 13: 'milk',
 14: 'seafood',
 15: 'rice',
 16: 'eggs',
 17: 'pork',
 18: 'legumes',
 19: 'butter',
 20: 'tree nuts & seeds',
 21: 'yogurt',
 22: 'turkey other poultry',
 23: 'non-food',
 24: 'meat',
 25: 'fish (farm-raised)'}

# Model 1


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") #Transforms text data into tensors that the model can compute
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #
accuracy = evaluate.load("accuracy")
id2label = fpg.id2label
label2id = fpg.label2id
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=int(len(id2label)), id2label=id2label, label2id=label2id
)
food_dataset = Dataset.from_pandas(fpg.data)
food_dataset = food_dataset.shuffle()
food_dataset = food_dataset.train_test_split(test_size=0.15)
tokenized_fds = food_dataset.map(preprocess_function, batched=True)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

Map:   0%|          | 0/68759 [00:00<?, ? examples/s]

Map:   0%|          | 0/12135 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="demo_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_fds["train"],
    eval_dataset=tokenized_fds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.257600,0.237701,0.936300
2,0.186500,0.219159,0.946271
3,0.130600,0.207127,0.952452
4,0.095400,0.227844,0.952204
5,0.073400,0.245293,0.953358
6,0.056800,0.261161,0.954100
7,0.042700,0.273615,0.956737
8,0.033200,0.286088,0.955913
9,0.029600,0.301430,0.955171
10,0.019800,0.307121,0.955583


TrainOutput(global_step=42980, training_loss=0.1172449538296131, metrics={'train_runtime': 1283.2714, 'train_samples_per_second': 535.81, 'train_steps_per_second': 33.493, 'total_flos': 3856282935220200.0, 'train_loss': 0.1172449538296131, 'epoch': 10.0})

In [ ]:
trainer.push_to_hub()

Cloning https://huggingface.co/anth0nyhak1m/demo_model into local empty directory.


Upload file pytorch_model.bin:   0%|          | 1.00/256M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.50k [00:00<?, ?B/s]

To https://huggingface.co/anth0nyhak1m/demo_model
   a7c5c94..bd6fd1b  main -> main

   a7c5c94..bd6fd1b  main -> main

To https://huggingface.co/anth0nyhak1m/demo_model
   bd6fd1b..914571b  main -> main

   bd6fd1b..914571b  main -> main



'https://huggingface.co/anth0nyhak1m/demo_model/commit/bd6fd1b4bf96566b01c0cb97487b5c7501f6cdd5'

# Model 2

Make sure tokenizer is working correctly

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") #Transforms text data into tensors that the model can compute
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #
accuracy = evaluate.load("accuracy")
id2label = pfpg.id2label
label2id = pfpg.label2id
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=int(len(id2label)), id2label=id2label, label2id=label2id
)
food_dataset = Dataset.from_pandas(pfpg.data)
food_dataset = food_dataset.shuffle()
food_dataset = food_dataset.train_test_split(test_size=0.15)
tokenized_fds = food_dataset.map(preprocess_function, batched=True)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

Map:   0%|          | 0/4551 [00:00<?, ? examples/s]

Map:   0%|          | 0/804 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_fds)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 19
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 4
    })
})


In [ ]:
tokenized_fds['train']['attention_mask']

[[1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [ ]:
training_args = TrainingArguments(
    output_dir="PFPC_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_fds["train"],
    eval_dataset=tokenized_fds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.266320,0.725124
2,1.641800,0.724653,0.842040
3,1.641800,0.565181,0.865672
4,0.503800,0.499650,0.876866
5,0.503800,0.470946,0.890547
6,0.242800,0.476914,0.896766
7,0.242800,0.476404,0.898010
8,0.125600,0.471585,0.906716
9,0.075800,0.471010,0.905473
10,0.075800,0.465293,0.904229


TrainOutput(global_step=2850, training_loss=0.46221350820440993, metrics={'train_runtime': 92.1037, 'train_samples_per_second': 494.117, 'train_steps_per_second': 30.943, 'total_flos': 243834446214450.0, 'train_loss': 0.46221350820440993, 'epoch': 10.0})

In [ ]:
trainer.push_to_hub()

Cloning https://huggingface.co/anth0nyhak1m/PFPC_model into local empty directory.


Upload file pytorch_model.bin:   0%|          | 1.00/256M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.50k [00:00<?, ?B/s]

To https://huggingface.co/anth0nyhak1m/PFPC_model
   06e6655..5cb49df  main -> main

   06e6655..5cb49df  main -> main

To https://huggingface.co/anth0nyhak1m/PFPC_model
   5cb49df..8cbfa0a  main -> main

   5cb49df..8cbfa0a  main -> main



'https://huggingface.co/anth0nyhak1m/PFPC_model/commit/5cb49df8cc44ee772b1b140bd2b0b60d4963d18c'

In [ ]:
for i in cleaned_datasets:
  print(i.name, len(i.data))

eggs 575
fish (farm-raised) 102
seafood 901
beef 2405
cheese 2310
turkey other poultry 1160
tree nuts & seeds 560
roots & tubers 4101
meals 7793
fruit 7534
grain products 5857
produce 43
legumes 807
meat 324
pork 1623
vegetables 9983
fish (wild) 198
beverages 4889
milk & dairy 1037
condiments & snacks 19493
rice 373
butter 212
milk 776
yogurt 913
chicken 2448
non-food 20


In [ ]:
ClnDataSet2id['bread']

KeyError: ignored

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") #Transforms text data into tensors that the model can compute
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #
accuracy = evaluate.load("accuracy")
tokenized_datasets = []

for food_df in cleaned_datasets:
  if len(food_df.data) == 1:
    tokenized_datasets.append(None)
  else:
    food_dataset = Dataset.from_pandas(food_df.data)
    food_dataset = food_dataset.shuffle()
    food_dataset = food_dataset.train_test_split(test_size=0.15)
    tokenized_fds = food_dataset.map(preprocess_function, batched=True)
    tokenized_datasets.append(tokenized_fds)

Map:   0%|          | 0/488 [00:00<?, ? examples/s]

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

Map:   0%|          | 0/86 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/765 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/137 [00:00<?, ? examples/s]

Map:   0%|          | 0/2080 [00:00<?, ? examples/s]

Map:   0%|          | 0/368 [00:00<?, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
for food_df in cleaned_datasets:
  if len(food_df.data) == 1:
    print(ClnDataSet2id[food_df.name], food_df.name)

In [ ]:
models_list =[]

for food_df in cleaned_datasets:
  print(food_df.name)
  if len(food_df.data) == 1:
    models_list.append(None)
    print(f"{food_df.name} model is None")
  else:
    id2label = food_df.id2label
    label2id = food_df.label2id
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=int(len(id2label)), id2label=id2label, label2id=label2id
    )
    models_list.append(model)

eggs


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

fish (farm-raised)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

seafood


...

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

chicken


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

non-food


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [ ]:
models_list[ClnDataSet2id['milk']]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
trainer_list = []

for food_df in cleaned_datasets:

  if len(food_df.data) == 1:
    models_list.append(None)
    print(f"{food_df.name} trainer is None")
  else:

    print(food_df.name)
    training_args = TrainingArguments(
        output_dir=f"CFGFP_{food_df.name}Calssifier_v1",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,
    )

    trainer = Trainer(
        model=models_list[ClnDataSet2id[food_df.name]],
        args=training_args,
        train_dataset=tokenized_datasets[ClnDataSet2id[food_df.name]]['train'],
        eval_dataset=tokenized_datasets[ClnDataSet2id[food_df.name]]["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

  trainer_list.append(trainer)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


eggs


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.073140,0.988506


fish (farm-raised)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.757535,0.982143


butter


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: ignored

In [ ]:
cleaned_datasets[ClnDataSet2id['seafood']].data

,text,labels
0,(X)RT SALMON ATLANTIC PORTION S/B 8OZ,11
1,POCTOPUS PHILL RAW,12
2,TUNA-CHUNK LIGHT-66.5oz. AMBROSIA,5
3,"TUNA, WHT ALBCR SOLID IN SPRG",5
4,KAJIKI SASHIMI GRADE,16
5,"SHRIMP, CKD 250-350 PLD FZN",3
6,FRESH ATL SALMON FIL,11
7,Azuma SL CK Tako Wild,12
8,SHRIMP 21/25 P&D WHT T/ON IQF,3
9,COD ATL PRTN BLSL IVP 6OZ ICEL,6


In [ ]:
ClnDataSet2id

{'produce': 0,
 'yogurt': 1,
 'meals': 2,
 'grain products': 3,
 'vegetables': 4,
 'milk': 5,
 'fruit': 6,
 'condiments & snacks': 7,
 'non-food': 8,
 'eggs': 9,
 'pork': 10,
 'tree nuts & seeds': 11,
 'legumes': 12,
 'meat': 13,
 'chicken': 14,
 'fish (farm-raised)': 15,
 'cheese': 16,
 'beverages': 17,
 'milk & dairy': 18,
 'butter': 19,
 'roots & tubers': 20,
 'bread': 21,
 'rice': 22,
 'beef': 23,
 'turkey & other poultry': 24,
 'seafood': 25,
 'fish (wild)': 26}

In [ ]:
len(cleaned_datasets[ClnDataSet2id['grain products']].data)

364

In [ ]:
ClnDataSet2id

{'fruit': 0,
 'meat': 1,
 'chicken': 2,
 'roots & tubers': 3,
 'milk': 4,
 'seafood': 5,
 'eggs': 6,
 'condiments & snacks': 7,
 'yogurt': 8,
 'milk & dairy': 9,
 'fish (wild)': 10,
 'vegetables': 11,
 'beverages': 12,
 'beef': 13,
 'fish (farm-raised)': 14,
 'non-food': 15,
 'produce': 16,
 'bread': 17,
 'grain products': 18,
 'butter': 19,
 'cheese': 20,
 'meals': 21,
 'pork': 22,
 'rice': 23,
 'turkey & other poultry': 24,
 'tree nuts & seeds': 25,
 'legumes': 26}

In [ ]:
models_list[ClnDataSet2id['grain products']]

In [ ]:
need_to_fix = {'milk', 'eggs', 'yogurt', 'produce', 'butter', 'bread', 'grain products', 'FOCUS ON THIS ONE: meals', }

In [ ]:
var = 'chicken'

training_args = TrainingArguments(
      output_dir=f"{cleaned_datasets[ClnDataSet2id[var]].name}",
      learning_rate=2e-5,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      num_train_epochs=1,
      weight_decay=0.01,
      evaluation_strategy="epoch",
      save_strategy="epoch",
      load_best_model_at_end=True,
      push_to_hub=False,
  )

trainer = Trainer(
      model=models_list[ClnDataSet2id[var]],
      args=training_args,
      train_dataset=tokenized_datasets[ClnDataSet2id[var]]['train'],
      eval_dataset=tokenized_datasets[ClnDataSet2id[var]]["test"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
  )

trainer.train()

RuntimeError: ignored

In [ ]:
from datasets import load_dataset
dataset = load_dataset('glue', 'mrpc', split='train')
from datasets import Value, ClassLabel

In [ ]:
tokenized_datasets[ClnDataSet2id['milk']]['train'].features

{'text': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
tokenized_datasets[ClnDataSet2id['meat']]['train'].features


{'text': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
new_features = tokenized_datasets[ClnDataSet2id['meat']]['train'].features.copy()

In [ ]:
tokenized_datasets[ClnDataSet2id['milk']]['train'].cast(new_features)

Casting the dataset:   0%|          | 0/72 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 72
})

In [ ]:
cleaned_datasets

NameError: ignored

SS Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") #Transforms text data into tensors that the model can compute
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #
accuracy = evaluate.load("accuracy")
id2label = dataset.id2label
label2id = dataset.label2id
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=int(len(id2label)), id2label=id2label, label2id=label2id
)
food_dataset = Dataset.from_pandas(dataset.data)
food_dataset = food_dataset.shuffle()
food_dataset = food_dataset.train_test_split(test_size=0.15)
tokenized_fds = food_dataset.map(preprocess_function, batched=True)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

Map:   0%|          | 0/68803 [00:00<?, ? examples/s]

Map:   0%|          | 0/12142 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="SS_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_fds["train"],
    eval_dataset=tokenized_fds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.153000,0.147161,0.952561
2,0.116500,0.137621,0.956185
3,0.095100,0.146163,0.959644
4,0.085100,0.154965,0.960221
5,0.070900,0.184759,0.959562
6,0.069000,0.202701,0.958574
7,0.059100,0.226634,0.958162
8,0.047000,0.211048,0.957256
9,0.039100,0.240539,0.957668
10,0.033300,0.286522,0.956597


TrainOutput(global_step=86020, training_loss=0.04787759229482793, metrics={'train_runtime': 2538.8484, 'train_samples_per_second': 542.002, 'train_steps_per_second': 33.882, 'total_flos': 7718544668639256.0, 'train_loss': 0.04787759229482793, 'epoch': 20.0})

In [ ]:
trainer.push_to_hub()

Cloning https://huggingface.co/anth0nyhak1m/SS_model into local empty directory.


Upload file pytorch_model.bin:   0%|          | 1.00/255M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.50k [00:00<?, ?B/s]

To https://huggingface.co/anth0nyhak1m/SS_model
   e54f243..0d24190  main -> main

   e54f243..0d24190  main -> main

To https://huggingface.co/anth0nyhak1m/SS_model
   0d24190..2e103fc  main -> main

   0d24190..2e103fc  main -> main



'https://huggingface.co/anth0nyhak1m/SS_model/commit/0d241908f22afb16049109cbf674ebc349feb553'